# Keras读写模型

In [1]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training data: x =', x_train.shape, ', y =', y_train.shape)
print('test data: x =', x_test.shape, ', y =', y_test.shape)

import keras

num_classes = 10

x_train = x_train.reshape(x_train.shape[0], -1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], -1).astype('float32') / 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('training data: x =', x_train.shape, ', y =', y_train.shape)
print('test data: x =', x_test.shape, ', y =', y_test.shape)

image_size = x_train.shape[-1]

from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(image_size,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

from keras.optimizers import *

from keras.losses import MSE

model.compile(loss=MSE,
              optimizer=Adagrad(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=1000,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

model.save(filepath='model.h5')

Using TensorFlow backend.


('training data: x =', (60000, 28, 28), ', y =', (60000,))
('test data: x =', (10000, 28, 28), ', y =', (10000,))
('training data: x =', (60000, 784), ', y =', (60000, 10))
('test data: x =', (10000, 784), ', y =', (10000, 10))
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 242,762
Trainable params: 242,762
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 1000

## 方式1：将模型（类似于TF中的graph对象）和网络参数参数写入h5文件中
Keras中的参数是根据名字导入的，此种方式需要提供网络代码
至此在本地文件夹中存在一个`model.h5`文件，里面保存着我们的模型参数<br>
保存模型参数：``model.save(filepath='model.h5')``<br>
导入模型参数：``model.load_weights(filepath='model.h5', by_name=True)``<br>

## 方式2：将模型写入json/yaml文件中
无需提供完整的网络代码<br>
保存模型（将模型写入json文件中）：
``
with open('model.json', 'w') as stream:
    stream.write(model.to_json())
``
<br>
导入模型（从json文件中构建模型）：<br>
``
from keras.models import model_from_json
with open('model.json', 'r') as stream:
    model = model_from_json(stream.read())
-# 从h5文件中导入网络参数
model.load_weights(filepath='model.h5', by_name=True)
-# 必须构建模型才能进行下一步训练或者测试
from keras.optimizers import *
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
``
<br>
yaml文件的读写方式与json类似

## 方式3：将网络参数写入numpy文件中
该方式的是通过调用keras的后台TF.Session将张量导出为numpy矩阵
``
parameters = [para for layer in model.layers for para in layer.weights]
parameters = keras.backend.get_session().run(parameters)
``